In [ ]:
pip install gymnasium[classic_control] ray[rllib] mediapy gputil

In [ ]:
import numpy as np
import ray
from ray import tune
import gymnasium
from mediapy import show_video


In [ ]:
env = gymnasium.make('MountainCarContinuous-v0', render_mode="rgb_array_list")
env.reset()
rewards = []
for _ in range(500):
    action = env.action_space.high
    obs, reward, _, _, _ = env.step(action)
    rewards.append(reward)
frames = env.render()

In [ ]:
show_video(frames)

In [ ]:
env, env.spec

In [ ]:
import ray
import gym
import ray.rllib.algorithms.ppo as ppo
from ray.tune.logger import pretty_print
from ray.tune.registry import register_env
# from gymnasium.wrappers import StepAPICompatibility
from gym.spaces import Box as GymBox
# from gymnasium.spaces import Box
from ray import tune
import torch
import os


In [ ]:

ray.init()

In [ ]:

def make_env(*args):
  print(f"Context is: {args}")
  return gym.make('MountainCarContinuous-v0')
  # return StepAPICompatibility(gym.make('MountainCarContinuous-v0'))


# def to_gym_box(b: Box):
#   return GymBox(b.low, b.high, b.shape, dtype=b.dtype)

register_env("custom_env", make_env)


config = ppo.DEFAULT_CONFIG.copy()
config["framework"] = "torch"
config["num_gpus"] = torch.cuda.device_count()
# we need one master
config["num_workers"] = 0
config["env"] = "custom_env"
# A HACK! In fact, this prevents rllib from working with gymnasium
# see line 2065 in rllib.evaluation.rollout_worker
# env = make_env()
# config["observation_space"] = to_gym_box(env.observation_space)
# config["action_space"] = to_gym_box(env.action_space)
# config["disable_env_checking"] = True

In [ ]:

tune_config = {
    "config": config,
    "stop": {"training_iteration": 2},
}

In [ ]:
tune.run(
    ppo.PPO, **tune_config
)

In [ ]:
env = make_env()

In [ ]:
env.observation_space.__class__

In [ ]:
pip show gym